## Questions: CASE
Use the SQL environment below to assist with answering the following questions. Whether you get stuck or you just want to double check your solutions, my answers can be found at the top of the next concept.

![schema](parch-and-posey-schema.png)

1. Write a query to display for each order, the account ID, total amount of the order, and the level of the order - ‘Large’ or ’Small’ - depending on if the order is $3000 or more, or smaller than $3000.


In [ ]:
SELECT account_id, total_amt_usd,
CASE WHEN total_amt_usd > 3000 THEN 'Large'
ELSE 'Small' END AS order_level
FROM orders;


2. Write a query to display the number of orders in each of three categories, based on the total number of items in each order. The three categories are: 'At Least 2000', 'Between 1000 and 2000' and 'Less than 1000'.


In [ ]:
SELECT CASE WHEN total >= 2000 THEN 'At Least 2000'
   WHEN total >= 1000 AND total < 2000 THEN 'Between 1000 and 2000'
   ELSE 'Less than 1000' END AS order_category,
COUNT(*) AS order_count
FROM orders
GROUP BY 1;


3. We would like to understand 3 different levels of customers based on the amount associated with their purchases. The top level includes anyone with a Lifetime Value (total sales of all orders) greater than 200,000 usd. The second level is between 200,000 and 100,000 usd. The lowest level is anyone under 100,000 usd. Provide a table that includes the level associated with each account. You should provide the account name, the total sales of all orders for the customer, and the level. Order with the top spending customers listed first.


In [ ]:
SELECT a.name, SUM(total_amt_usd) total_spent, 
     CASE WHEN SUM(total_amt_usd) > 200000 THEN 'top'
     WHEN  SUM(total_amt_usd) > 100000 THEN 'middle'
     ELSE 'low' END AS customer_level
FROM orders o
JOIN accounts a
ON o.account_id = a.id 
GROUP BY a.name
ORDER BY 2 DESC;



4. We would now like to perform a similar calculation to the first, but we want to obtain the total amount spent by customers only in 2016 and 2017. Keep the same levels as in the previous question. Order with the top spending customers listed first.


In [ ]:
select a.name, sum(o.total_amt_usd),
case
	when sum(o.total_amt_usd) > 200000 then 'Larger'
    when sum(o.total_amt_usd) > 100000 and sum(o.total_amt_usd) < 200000 then 'Medium'
    else 'smaller'
end clevel
from accounts a
	join orders o
    	on o.account_id = a.id
where extract(year from occurred_at) in (2016, 2017)
group by 1
order by 2 desc;

|id	|sum	|clevel|
|---|-------|------|
|4251|	255319.18|	Larger|
|4151|	172180.04|	Medium|
1831	163471.78	Medium
4161	148105.93	Medium
1521	129157.38	Medium
2911	121873.16	Medium
2101	114003.21

5. We would like to identify top performing sales reps, which are sales reps associated with more than 200 orders. Create a table with the sales rep name, the total number of orders, and a column with top or not depending on if they have more than 200 orders. Place the top sales people first in your final table.

In [ ]:
select s.name, count(o.id) num_of_orders,
	case
    	when count(o.id) > 200 then 'top'
        else 'not'
    end performance
from sales_reps s
	join accounts a on a.sales_rep_id = s.id
    join orders o on o.account_id = a.id
group by 1
order by 2 desc;


name	num_of_orders	performance
Earlie Schleusner	335	top
Vernita Plump	299	top
Tia Amato	267	top
Georgianna Chisholm	256	top
Moon Torian	250	top


6. The previous didn't account for the middle, nor the dollar amount associated with the sales. Management decides they want to see these characteristics represented as well. We would like to identify top performing sales reps, which are sales reps associated with more than 200 orders or more than 750000 in total sales. The middle group has any rep with more than 150 orders or 500000 in sales. Create a table with the sales rep name, the total number of orders, total sales across all orders, and a column with top, middle, or low depending on this criteria. Place the top sales people based on dollar amount of sales first in your final table. You might see a few upset sales people by this criteria!



In [ ]:
select s.name, count(o.id) num_of_orders, sum(total_amt_usd) total_sales,
	case
    	when count(o.id) >= 200 or sum(total_amt_usd) >= 750000 then 'top'
        when (count(o.id) between 150 and 199) or (sum(total_amt_usd) >= 500000) then 'middle'
        else 'low'
    end performance
from sales_reps s
	join accounts a on a.sales_rep_id = s.id
    join orders o on o.account_id = a.id
group by 1
order by 3 desc;


Output50 results
name	num_of_orders	total_sales	performance
Earlie Schleusner	335	1098137.72	top
Tia Amato	267	1010690.60	top